In [1]:
# Standard imports
import os
import torch
from tqdm import tqdm
import plotly.express as px
import gc
from utils import load_our_sae, get_tokens
from sae_vis.data_config_classes import SaeVisConfig
from sae_vis.data_storing_fns import SaeVisData
os.environ["TOKENIZERS_PARALLELISM"] = "false"
torch.set_grad_enabled(False)

hook_point_layer=1
hook_point=f"blocks.{hook_point_layer}.attn.hook_z"

In [2]:
hook_point_head_index = 6
model, our_sae, activations_loader = load_our_sae(hook_point_head_index=hook_point_head_index)
all_tokens = get_tokens(activations_loader)

Loading SAE for head # 6
Loaded pretrained model gelu-2l into HookedTransformer
Moving model to device:  cpu


Resolving data files:   0%|          | 0/28 [00:00<?, ?it/s]

Loading dataset shards:   0%|          | 0/24 [00:00<?, ?it/s]

100%|██████████| 1024/1024 [00:26<00:00, 38.42it/s]


In [3]:
gc.collect(); torch.cuda.empty_cache()
# WARNING: ALWAYS MAKE SURE TO CHECK hook_point_head_index in the cell above. 
test_feature_idx_gpt = list(range(3, 4))

# setting batch_size = 8192 and minibatch_size_tokens = 32 avoids OOMS with A100 GPUs.
feature_vis_config_gpt = SaeVisConfig(
    hook_point=hook_point,
    hook_point_head_index=hook_point_head_index,
    features=test_feature_idx_gpt,
    batch_size=2, 
    minibatch_size_tokens=16,
    verbose=True,
)

sae_vis_data_gpt = SaeVisData.create(
    encoder=our_sae,
    model=model, # type: ignore
    tokens=all_tokens,  # type: ignore
    cfg=feature_vis_config_gpt,
)

gc.collect(); torch.cuda.empty_cache(); gc.collect()

output_dir = f"./dashboards/head{hook_point_head_index}"
os.makedirs(output_dir, exist_ok=True)
for feature in test_feature_idx_gpt:
    filename = os.path.join(output_dir, f"{feature}.html")
    sae_vis_data_gpt.save_feature_centric_vis(filename, feature)
    # display_vis_inline(filename)

Forward passes to cache data for vis:   0%|          | 0/1 [00:00<?, ?it/s]

Extracting vis data from cached data:   0%|          | 0/1 [00:00<?, ?it/s]

num chunks: 1, chunk size: 1000
torch.Size([1, 2048]) torch.Size([2350])


┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━┳━━━━━━━┓
┃ Task                                           ┃ Time  ┃ Pct % ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━╇━━━━━━━┩
│ (1) Initialization                             │ 0.03s │ 3.2%  │
│ (2) Forward passes to gather model activations │ 0.78s │ 75.0% │
│ (3) Computing feature acts from model acts     │ 0.01s │ 0.8%  │
│ (4) Getting data for tables                    │ 0.02s │ 1.7%  │
│ (5) Getting data for histograms                │ 0.02s │ 1.5%  │
│ (6) Getting data for sequences                 │ 0.17s │ 16.2% │
│ (7) Getting data for quantiles                 │ 0.02s │ 1.6%  │
└────────────────────────────────────────────────┴───────┴───────┘

Saving feature-centric vis:   0%|          | 0/1 [00:00<?, ?it/s]